In [6]:
# load and normalize CIFAR10
import torch
import torchvision
import torchvision.transforms as transforms
#%matplotlib inline
#The output of torchvision datasets are PILImage images of range [0, 1].
#We transform them to Tensors of normalized range [-1, 1].
transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5) ,(0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                       download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                         shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                        shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
# training on GPU
# Define a convolution Neural network
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 *5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if device == "cuda:0" and torch.cuda.device_count() > 1:
    print("lets use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)
print(device)
net.to(device)
# Define a Loss function and optimizer
# use a Classification Cross-Entropy loss and SGD with momentum
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(1000): # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        # make tensors runnable on GPU
        inputs , labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss:%.3f'  % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

Files already downloaded and verified
Files already downloaded and verified
cuda:0
[1,  2000] loss:2.217
[1,  4000] loss:1.863
[1,  6000] loss:1.676
[1,  8000] loss:1.568
[1, 10000] loss:1.532
[1, 12000] loss:1.468
[2,  2000] loss:1.387
[2,  4000] loss:1.350
[2,  6000] loss:1.326
[2,  8000] loss:1.311
[2, 10000] loss:1.297
[2, 12000] loss:1.279
[3,  2000] loss:1.204
[3,  4000] loss:1.181
[3,  6000] loss:1.201
[3,  8000] loss:1.170
[3, 10000] loss:1.132
[3, 12000] loss:1.173
[4,  2000] loss:1.082
[4,  4000] loss:1.087
[4,  6000] loss:1.094
[4,  8000] loss:1.070
[4, 10000] loss:1.070
[4, 12000] loss:1.046
[5,  2000] loss:1.002
[5,  4000] loss:0.977
[5,  6000] loss:0.996
[5,  8000] loss:1.016
[5, 10000] loss:1.010
[5, 12000] loss:0.997


Process Process-21:
Process Process-22:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubun

KeyboardInterrupt: 

KeyboardInterrupt
  File "/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 115, in default_collate
    return torch.stack(batch, 0, out=out)
KeyboardInterrupt
